### Graded Lab 4

Hello ! Welcome to Graded Lab of Module 4.

In the last assignment we had worked on correlation , hypothesis which business wanted us to test.
Its high time when we look at the missing values & outliers stuff.
**Here we will work on missing values & outliers.**

In case if you are not able to recollect the problem description and data description then mentioning it below.

Lets look at the problem statement,

*Client: ABC Retail, Incorporated, rest-of-the-world division* 

***Project name: Online retail sales analysis*** 

An online retailer, ABC, Inc., operates in nearly 100 countries worldwide, selling furniture, office supplies and technology products to customers in three segments: consumer, corporate and home office. ABC, Inc. is a US-based company, and it has two major divisions: US and rest of the world. We are working with the rest of the world division of the company. 

They have provided us with online sales transaction data from 2011 to 2014.

We are given 3 datasets:-

1. Data on each sale; 51290 records; all data in US dollars
It contains fields like
**order_id** (identifier) ,order_date ,ship_date ,ship_mode ,**customer_id**(identifier) ,product_id ,category ,sub_category ,product_name ,sales ,quantity ,discount ,profit ,shipping_cost ,order_priority ,**vendor_code** (identifier) 


2. Data on the customers; 1590 records 
It contains fields like
**customer_id** (identifier) ,customer_name ,city ,state ,country ,postal_code ,segment ,market ,region 

3. Data on vendors who supply the retailer; 65 records 
It contains fields like
vendor ,**vendor_code** (identifier) 

We need to analyze the data and need to provide answer to different questions asked by company officials.

In [ ]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from itertools import combinations
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

### Reading sales data
sales = pd.read_csv('sales_data_M4.csv')

### Reading customer data
cust = pd.read_csv(r'customers.csv',encoding='iso-8859-1')

### Reading vendor data
vend = pd.read_csv(r'vendors.csv')

sales.head()

In [ ]:
pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format','{:.2f}'.format)

In [ ]:
cust.head()

In [ ]:
vend.head()

In order to solve the next questions , we need to combine all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. So here we have written a data processing function.
There are 2 tasks which are to be performed.
1. Merge/ Join all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. (Understand which should be the joining key , type of join , refer .merge() function of pandas)
2. Convert 'order_date' into a datetime column.
**Return output as a dataframe**

In [ ]:
#### data_merging & order_date processing , data1 will be sales , data2 will be customer dataset & data3 will be vendor dataset.

def data_process(data1,data2,data3):
    df = data1.merge(right=data3, how="left", on="vendor_code")
    df2 = df.merge(right=data2, how="left", on="customer_id")
    df2["order_date"] = pd.to_datetime(df2['order_date'], format='%d/%m/%Y')
    return df2

In [ ]:
sales= data_process(data1=sales.copy(),data2=cust.copy(),data3=vend.copy())

In [ ]:
assert sales['order_date'].dtypes=='<M8[ns]' ,'Make sure if you have converted order_date into a datetime format correctly or not.'
assert sales.shape== (51290,26) ,'Checking size and shape of dataframe after merging is a very important check.'

In [ ]:
sales.columns

Note that in this dataset we are purposefully introducing some of the missing values.

### Q.1 Return column names & missing values percentage for the columns which have % misisng value >50. Output should be a dictionary. For eg:- { column_name : %}, make sure to round off percentages to 2 decimals.

In [ ]:
totalna = sales.isnull().sum().sum()
totalna

In [ ]:
missingdf = pd.DataFrame(sales.isnull().sum(), columns=["no"]).reset_index()
missingdf

In [ ]:
missingdf["%"] = round((missingdf["no"]/totalna * 100),2)

In [ ]:
missingdf

In [ ]:
missingdf.loc["postal_code"]["%"]

### ----------------------------------------------------------

In [ ]:
def missing_value_col(data):
    # your code here
    # Calculate missing value percentages for each column
    missing_percentage = (data.isnull().sum() / len(data)) * 100
    
    # Filter columns with missing values > 50%
    missing_columns = missing_percentage[missing_percentage > 50]
    
    # Round off percentages to 2 decimals
    missing_columns = missing_columns.round(2)

    # Convert the result to a dictionary
    missing_columns_dict = missing_columns.to_dict()
    
    return missing_columns_dict

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell

In [ ]:
missing_value_col(data=sales)

#### **Senior data scientists after consulting with the business have decided that such columns needs to be dropped before going further.**

In [ ]:
high_missing_col=list(missing_value_col(data=sales).keys()) #Enter the columns with high % of missing values
### We will drop columns with such high missing values
sales.drop(high_missing_col,axis=1,inplace=True)

In [ ]:
sales.head()

### Q.2 Fill the missing data in 'category' column with appropriate method.  
(Hint:- Remember variable 'category' & 'sub-category' are related & hierachy goes like this category -> sub-category)

Approach:-

1. Create a key-pair dictionary where key will be product category & pair values will be sub-categories.

2. Wherever category value is missing check for corresponding sub-category value. 

3. From sub-category value trace back its original product category value by looking in dictionary, get a product value.

4. Fill the blank product category value by the same.

In [ ]:
### create a dictionary where keys will be categories & sub-categories as values. store in inside Category_dict.

def cat_dict(data,main_col,sub_col):    
    Category_dict = {}
    for index, row in data.iterrows():
        if not pd.isna(row[main_col]) and not pd.isna(row[sub_col]):
            Category_dict[row[main_col]] = Category_dict.get(row[main_col], []) + [row[sub_col]]
    return Category_dict

def category_subcategory_map_dataframe(df, Category_dict, main_col, sub_col):
    def category_subcategory_map(row):
        if pd.isna(row[main_col]):  # If the main column is null
            if not pd.isna(row[sub_col]):  # If the sub column is not null
                for k, v in Category_dict.items():
                    if row[sub_col] in Category_dict[k]:
                        return k  # k is the correct sector for the category value v
        else:
            return row[main_col]  # If the main column is not null, return its existing value

    df[main_col] = df.apply(category_subcategory_map, axis=1)
    return df[main_col]

In [ ]:
Category_dict=cat_dict(data=sales,main_col='category',sub_col='sub_category')
sales['category']=category_subcategory_map_dataframe(df=sales, Category_dict=Category_dict, main_col='category', sub_col='sub_category')

In [ ]:
sales.category.isnull().sum()

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell

### Q.3 ) Fill the missing values in sales column by mean. Return output series.

In [ ]:
def fill_missing(data,col):
    mean_value = data[col].mean()
    data[col] = data[col].fillna(mean_value)
    return data[col]

In [ ]:
sales['sales']=fill_missing(data=sales,col='sales')

In [ ]:
sales.sales.isnull().sum()

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell

### Q.4 ) How many orders have sales falling in the following groups 
Groups :- [0,12),[12,37),[37,113),[113,186),[186,1200),[1200,max).

**Note :-[a,b) indicates that bin includes a but excudes b]**

**Return output series**

In [ ]:
sales["sales"].describe()

In [ ]:
def manual_bin(data,col):
    # Define the bin edges
    bins = [0, 12, 37, 113, 186, 1200, float('inf')]
    
    # Use the cut function to categorize 'sales' into the specified bins
    data['bin'] = pd.cut(data[col], bins, right=False)
    
    # Use value_counts to count the number of orders in each bin
    bin_counts = data['bin'].value_counts().sort_index()

    # Reset the index to have a clean output series
    bin_counts = bin_counts.reset_index()

    # Rename the columns for clarity
    # bin_counts.columns = ['Sales Range', 'Number of Orders']
    bin_counts = pd.Series(data=bin_counts["bin"])
    
    return bin_counts

In [ ]:
manual_bin(sales, "sales")

In [ ]:
assert type(manual_bin(data=sales,col='sales'))==pd.Series, "Please provide output in series format."

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell

### Q.5 )  Sometimes, it is difficult to comprehend the sales value as a numeric value. So, the business team wants to observe the sales as Low, Medium and High categories.  Write a code to discretize the sales data and  return a series containing Low, Medium and High with the respective number of records.

In [ ]:
sales.quantity.value_counts()

In [ ]:
def discrete_bin(data,col):
    # Define the bin edges and labels
    bins = [1, 5, 10, 14]
    labels = ['Low', 'Medium', 'High']
    
    # Use the cut function to categorize 'sales' into the specified bins and labels
    data['category'] = pd.cut(data[col], bins, labels=labels)
    
    # Use value_counts to count the number of records in each category
    category_counts = data['category'].value_counts().sort_index()
    
    # Reset the index to have a clean output series
    category_counts = category_counts.reset_index()
    
    category_counts = pd.Series(data=category_counts["category"])
    
    return category_counts

In [ ]:
discrete_bin(sales,'quantity')

In [ ]:
assert type(discrete_bin(data=sales,col='quantity'))==pd.Series, "Please provide output in series format."

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell

### Q.6. In order to perform further analysis, we intend to utilize the 'order_priority' variable, which has categorical values ('First Class'(0) > 'Second Class'(1) > 'Standard Class'(2) > 'Same Day'(3)). To facilitate this analysis, we need to convert 'order_priority' into a numerical feature. Please encode 'order_priority' into a numerical format.Return output series.

In [ ]:
sales.ship_mode.value_counts()

In [ ]:
## Select Appropriate priority

priority= ['Standard Class', 'Second Class', 'First Class', 'Same Day']

# Mapping dictionary to encode 'order_priority'
order_priority_mapping = {
    'Same Day': 3,
    'First Class': 0,
    'Second Class': 1,
    'Standard Class': 2  }

def transforming_ordered_var(data,col,priority,nan_val):
    data[col] = data[col].map(order_priority_mapping)
    data[col] = data[col].fillna(nan_val)
    return data[col]

In [ ]:
transforming_ordered_var(sales,'ship_mode',priority,5)

In [ ]:
sales['shipping_mode_encoded']=transforming_ordered_var(data=sales,col='ship_mode',priority=priority,nan_val=5)

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell

### Q.8) Outliers can be defined in different ways. For each of the following definitions of outliers, what percent of the values in the  'sales' are considered outliers? Give the answer as a percent rounded upto two decimal places.

Note -
1. 'IQR_detection' : Less than (Q1 - 1.5*IQR) and greater than (Q3 + 1.5*IQR) are considered outliers

2. 'percentile_detection': Below the 3rd percentile and above the 97th percentile are considered outliers

3. 'mean_SD' : Less than (mean - 3*SD) and greater than (mean + 3*SD) are considered outliers 

In [ ]:
sales["sales"]

In [ ]:
def detect_outliers(data,col,method):
    if method=='IQR_detection':
        # Calculate the first quartile (Q1) and the third quartile (Q3)
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        
        # Calculate the interquartile range (IQR)
        IQR = Q3 - Q1
        
        # Determine the lower and upper bounds for outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Count the number of outliers
        outliers = (data[col] < lower_bound) | (data[col] > upper_bound)
        
    elif method=='percentile_detection':
        
        # Calculate the 3rd and 97th percentiles
        percentile_3 = data[col].quantile(0.03)
        percentile_97 = data[col].quantile(0.97)
        
        # Determine the lower and upper bounds for outliers
        lower_bound = percentile_3
        upper_bound = percentile_97
        
        # Count the number of outliers
        outliers = (data[col] < lower_bound) | (data[col] > upper_bound)
        
    elif method=='mean_SD':
        # Calculate the mean and standard deviation (SD)
        mean = data[col].mean()
        std_dev = data[col].std()
        
        # Determine the lower and upper bounds for outliers
        lower_bound = mean - 3 * std_dev
        upper_bound = mean + 3 * std_dev
        
        # Count the number of outliers
        outliers = (data[col] < lower_bound) | (data[col] > upper_bound)
        
    outliers_perc = float((outliers.sum() / len(data[col])) * 100)
    
            
    return outliers_perc

# Assert the type of the output
# assert type(detect_outliers(data=data['sales'], col='sales', method='IQR_detection')) == np.float

In [ ]:
detect_outliers(sales,'sales','IQR_detection')

In [ ]:
assert type(detect_outliers(data=sales,col='sales',method='IQR_detection'))==np.float, 'Make sure that output is returned as float'

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell

### Q.10) Detect upper end outliers in profit column by 'IQR_detection'. Return number of upper end outliers.

In [ ]:
def detect_upperend_outliers(data,col):
    
    # Calculate the first quartile (Q1) and the third quartile (Q3)
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
        
    # Calculate the interquartile range (IQR)
    IQR = Q3 - Q1
        
    # Determine the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
        
    # Count the number of outliers
    upper_outliers = int((data[col] > upper_bound).sum())
        
    return upper_outliers

In [ ]:
detect_upperend_outliers(sales,'profit')

In [ ]:
assert type(detect_upperend_outliers(data=sales,col='profit'))==int , 'No of outliers should be an integer , make sure that you are returning output in integer format.'

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell

### Q.11) Floor & Capping outliers at 97% at 3%. Return the processed column as output.

In [ ]:
sales.head()

In [ ]:
def cap_and_floor_column(column, cap_percentile, floor_percentile):
    # Calculate the 3rd and 97th percentiles
    percentile_3 = column.quantile(0.03)
    percentile_97 = column.quantile(0.97)
        
    # Determine the lower and upper bounds for outliers
    lower_bound = percentile_3
    upper_bound = percentile_97
        
    # Cap and floor the column based on the bounds
    capped_and_floored_column = column.clip(lower=lower_bound, upper=upper_bound)
    
    return capped_and_floored_column ## Capped ,Floored series

In [ ]:
sales['profit_outlier_capped'] = cap_and_floor_column(column=sales['profit'], cap_percentile=97, floor_percentile=3)

In [ ]:
sales['profit_outlier_capped']

In [ ]:
sales

In [ ]:
assert round(np.quantile(sales['profit_outlier_capped'],0.97),4)==301.9693, 'Make sure that you are flooring and capping with correct percentiles & returning the correct output.'
assert round(np.quantile(sales['profit_outlier_capped'],0.03),4)==-144.632, 'Make sure that you are flooring and capping with correct percentiles & returning the correct output.'

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell

### Q.12) Business team is interested in knowing how many vendors have profit_outlier_capped >10000 & vendors with profit_outlier_capped <10000. Return the dataframe with 3 columns vendor  , profit_outlier_capped , profit_bins & sort it with 'profit_outlier_capped' in descending order.

Instructions:-
1. Bins should be in (a,b] format. [a,b) indicates that bin includes a but excudes b])
2. Make sure that for 1st interval it is **left inclusive** i.e [a,b].
3. Segment the ''profit_outlier_capped'' variable in 2 bins. (first bin with profit>10000 ,second bin with profit<10000).

In [ ]:
sales.head()

In [ ]:
sales.groupby("vendor").mean()

In [ ]:
# Create two bins: one for profit > 10000 and one for profit <= 10000
bins = [0, 10000, float('inf')]
    
# Create labels for the bins
labels = ['<= 10000', '> 10000']

In [ ]:
# Create a new column 'profit_bins' that segments 'profit_outlier_capped'
sales['profit_bins'] = pd.cut(sales['profit_outlier_capped'], bins=bins, labels=labels, right=False)

In [ ]:
sales.head()

In [ ]:
# Filter the dataframe to include only 'vendor', 'profit_outlier_capped', and 'profit_bins' columns
sales = sales[['vendor', 'profit_outlier_capped', 'profit_bins']]
sales

In [ ]:
sales['vendor'].value_counts()

In [ ]:
sales

In [ ]:
def profit_vendor(data,col):
    # Create two bins: one for profit > 10000 and one for profit <= 10000
    bins = [0, 10000, float('inf')]
    
    # Create labels for the bins
    labels = ['<= 10000', '> 10000']
    
    # Create a new column 'profit_bins' that segments 'profit_outlier_capped'
    data['profit_bins'] = pd.cut(data[col], bins=bins, labels=labels, right=False)
    
    # Filter the dataframe to include only 'vendor', 'profit_outlier_capped', and 'profit_bins' columns
    data = data[['vendor', 'profit_outlier_capped', 'profit_bins']]
    
    # Drop duplicate values in the 'vendor' column to make it distinct
    data['vendor'] = data['vendor'].drop_duplicates()
    
        
    # Reset the index to have a clean result
    data.reset_index(drop=True, inplace=True)
    
    # Sort the dataframe by 'profit_outlier_capped' in descending order
    data = data.sort_values(by='profit_outlier_capped', ascending=False)
    
    data = pd.DataFrame(data)
    
    return data

In [ ]:
assert type(profit_vendor(data=sales,col='profit_outlier_capped'))==pd.DataFrame ,'Make sure that your output is a dataframe '
assert profit_vendor(data=sales,col='profit_outlier_capped').shape==(65,3) ,'Make sure that shape of your dataframe is correct'

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell

#  55/100 points earned

# 